In [2]:
!pip install pyspark
# Get a Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cloudanum').getOrCreate()

     ---------------------------------------- 0.0/310.8 MB ? eta -:--:--
     --------------------------------------- 0.7/310.8 MB 21.1 MB/s eta 0:00:15
     --------------------------------------- 1.4/310.8 MB 18.2 MB/s eta 0:00:17
     --------------------------------------- 2.1/310.8 MB 16.6 MB/s eta 0:00:19
     --------------------------------------- 2.9/310.8 MB 16.8 MB/s eta 0:00:19
     --------------------------------------- 3.7/310.8 MB 17.0 MB/s eta 0:00:19
      -------------------------------------- 4.5/310.8 MB 16.9 MB/s eta 0:00:19
      -------------------------------------- 5.1/310.8 MB 15.6 MB/s eta 0:00:20
      -------------------------------------- 5.9/310.8 MB 16.3 MB/s eta 0:00:19
      -------------------------------------- 6.8/310.8 MB 16.6 MB/s eta 0:00:19
      -------------------------------------- 7.6/310.8 MB 16.7 MB/s eta 0:00:19
     - ------------------------------------- 8.5/310.8 MB 17.0 MB/s eta 0:00:18
     - ------------------------------------- 9.

RuntimeError: Java gateway process exited before sending its port number